### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [6]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

#### 1. 웹페이지 분석 : URL

In [4]:
query = '삼성전자'
url = f'https://search.naver.com/search.naver?query={query}'
url

'https://search.naver.com/search.naver?query=삼성전자'

#### 2. request(URL) > response : str(html)

In [8]:
response = requests.get(url)
response

<Response [200]>

In [9]:
response.text[:200]

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" c'

#### 3. str(html) > bs object

In [13]:
docm = BeautifulSoup(response.text, 'html.parser')
type(docm)

bs4.BeautifulSoup

#### 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [14]:
dir(docm)

['ASCII_SPACES',
 'DEFAULT_BUILDER_FEATURES',
 'DEFAULT_INTERESTING_STRING_TYPES',
 'EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'NO_PARSER_SPECIFIED_WARNING',
 'ROOT_TAG_NAME',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_clone',
 '_decode_markup',
 '_event_stream',
 '_feed',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descenda

In [15]:
help(docm.select)

Help on method select in module bs4.element:

select(selector, namespaces=None, limit=None, **kwargs) method of bs4.BeautifulSoup instance
    Perform a CSS selection operation on the current element.
    
    This uses the SoupSieve library.
    
    :param selector: A string containing a CSS selector.
    
    :param namespaces: A dictionary mapping namespace prefixes
       used in the CSS selector to namespace URIs. By default,
       Beautiful Soup will use the prefixes it encountered while
       parsing the document.
    
    :param limit: After finding this number of results, stop looking.
    
    :param kwargs: Keyword arguments to be passed into SoupSieve's
       soupsieve.select() method.
    
    :return: A ResultSet of Tags.
    :rtype: bs4.element.ResultSet



In [18]:
elements = docm.select('.lst_related_srch > .item')
len(elements)

10

#### 5. str(text) > DataFrame

In [22]:
element = elements[0]
keyword = element.select_one('.tit').text
keyword

'삼성전자주가'

In [24]:
keywords = [element.select_one('.tit').text for element in elements]
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자서비스',
 '삼성전자주식',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자몰',
 '삼성 전자레인지',
 '삼성전자 채용',
 '삼성전자 고객센터']

In [28]:
# 데이터 프레임 만들기
df = pd.DataFrame({'keyword': keywords})
df['query'] = query
df

,keyword,query
0,삼성전자주가,삼성전자
1,삼성전자 배당금,삼성전자
2,삼성전자서비스,삼성전자
3,삼성전자주식,삼성전자
4,오늘 삼성전자 주가,삼성전자
5,삼성전자 배당금 지급일,삼성전자
6,삼성전자몰,삼성전자
7,삼성 전자레인지,삼성전자
8,삼성전자 채용,삼성전자
9,삼성전자 고객센터,삼성전자


In [29]:
from datetime import datetime

now = datetime.now().strftime('%Y-%m-%d %H:%M')
df['datetime'] = now
df

,keyword,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:20
1,삼성전자 배당금,삼성전자,2023-08-25 12:20
2,삼성전자서비스,삼성전자,2023-08-25 12:20
3,삼성전자주식,삼성전자,2023-08-25 12:20
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:20
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:20
6,삼성전자몰,삼성전자,2023-08-25 12:20
7,삼성 전자레인지,삼성전자,2023-08-25 12:20
8,삼성전자 채용,삼성전자,2023-08-25 12:20
9,삼성전자 고객센터,삼성전자,2023-08-25 12:20


In [30]:
def naver_relate_keywords(query):
    url = f'https://search.naver.com/search.naver?query={query}'
    response = requests.get(url)
    docm = BeautifulSoup(response.text, 'html.parser')
    elements = docm.select('.lst_related_srch > .item')
    keywords = [element.select_one('.tit').text for element in elements]
    
    df = pd.DataFrame({'keyword': keywords})
    df['query'] = query
    now = datetime.now().strftime('%Y-%m-%d %H:%M')
    df['datetime'] = now
    return df

In [31]:
naver_relate_keywords('삼성전자')

,keyword,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:23
1,삼성전자 배당금,삼성전자,2023-08-25 12:23
2,삼성전자서비스,삼성전자,2023-08-25 12:23
3,삼성전자주식,삼성전자,2023-08-25 12:23
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:23
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:23
6,삼성전자몰,삼성전자,2023-08-25 12:23
7,삼성 전자레인지,삼성전자,2023-08-25 12:23
8,삼성전자 채용,삼성전자,2023-08-25 12:23
9,삼성전자 고객센터,삼성전자,2023-08-25 12:23


In [32]:
naver_relate_keywords('카카오')

,keyword,query,datetime
0,카카오뱅크,카카오,2023-08-25 12:23
1,카카오웹툰,카카오,2023-08-25 12:23
2,카카오게임즈,카카오,2023-08-25 12:23
3,카카오페이,카카오,2023-08-25 12:23
4,카카오맵,카카오,2023-08-25 12:23
5,카카오톡,카카오,2023-08-25 12:23
6,카카오톡 이모티콘샵,카카오,2023-08-25 12:23
7,카카오 이모티콘샵,카카오,2023-08-25 12:23
8,카카오프렌즈샵,카카오,2023-08-25 12:23
9,카카오뱅크 주가,카카오,2023-08-25 12:23


In [35]:
naver_relate_keywords('스마트팩토리')

,keyword,query,datetime
0,스마트팩토리 관련주,스마트팩토리,2023-08-25 12:23
1,아산스마트팩토리마이스터고등학교,스마트팩토리,2023-08-25 12:23
2,sk온 스마트팩토리,스마트팩토리,2023-08-25 12:23
3,sk 스마트팩토리,스마트팩토리,2023-08-25 12:23
4,스마트팩토리 지원사업,스마트팩토리,2023-08-25 12:23
5,스마트팩토리 교육,스마트팩토리,2023-08-25 12:23
